# Convert Units
Pull aboveground biomass data from Chloris and convert to aboveground carbon density by multiplying the data by 0.476.

### Import required packages.

In [1]:
import snowflake.connector
import yaml
import pandas as pd
import matplotlib.pyplot as plt

### Connect to snowflake.
Note that the access credentials to the snowflake database are not included below. The code should be modified where shown to include the access credentials provided to you. 

In [2]:
# Pull database information from the config yaml file. 
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

# Connect to the snowflake database using info from config file.
conn = snowflake.connector.connect(
    account = config['database-info']['account'], # Replace 'config['database-info']['account']' with the database account name. 
    user = config['database-info']['user'], # Replace 'config['database-info']['user']' with the database username.
    password = config['database-info']['password'], # Replace 'config['database-info']['password']' with the database password.
    warehouse = config['database-info']['warehouse'], # Replace 'config['database-info']['warehouse']' with the database warehouse name.
    database = config['database-info']['database'], # Replace 'config['database-info']['database']' with the database name.
)

### Pull the data of interest.
The SQL query below demonstrates how to access variables from the Planet Forest Carbon dataset, converting from aboveground carbon density to aboveground biomass and renaming variables for the year 2020.

**SQL Query Breakdown**

We begin by selecting the variables of interest from the Planet Forest Carbon dataset. In addition to the year, lat, and lon values, we pull the raw aboveground_carbon_density data, and rename this variable 'planet_acd' using the SQL clause 'AS'. Then, we convert this variable to aboveground biomass by dividing by 0.476, and, like above, renaming the resulting variable 'planet_agb'.
```SQL
    SELECT
        year,
        lon,
        lat,
        aboveground_carbon_density AS planet_acd,
        aboveground_carbon_density / 0.476 AS planet_agb
```

Next, we use the SQL clause 'FROM' to indicate which dataset the variables should be pulled from.
```SQL
    FROM
        planet.pv_forest_carbon
```

Finally, we select only the rows of the dataset where the year is equal to 2020 using the SQL clause 'WHERE'. 
```SQL
    WHERE
        c.year = 2020
```

In [3]:
# Define the SQL query to get Planet data for 2020.
sql_query = """
    SELECT
        year,
        lon,
        lat,
        aboveground_carbon_density AS planet_acd,
        aboveground_carbon_density / 0.476 AS planet_agb
    FROM
        planet.pv_forest_carbon
    WHERE
        year = 2020
    """

# Execute the query and get the rows and columns of data.
cursor = conn.cursor()
cursor.execute(sql_query)
data_rows = cursor.fetchall()
data_columns = [desc[0] for desc in cursor.description]
cursor.close()
conn.close()

### Convert to dataframe.

In [4]:
# Format the data into a pandas dataframe.
data_df = pd.DataFrame(data_rows, columns=data_columns)
data_df.head(5)

,YEAR,LON,LAT,PLANET_ACD,PLANET_AGB
0,2020,-2.77862500,57.12337500,0E-8,0E-12
1,2020,-2.77912500,57.12262500,49.00000000,102.941176470588
2,2020,-2.77962500,57.12212500,42.00000000,88.235294117647
3,2020,-2.77887500,57.12212500,53.00000000,111.344537815126
4,2020,-2.77987500,57.12187500,0E-8,0E-12
